### Imports

In [ ]:
import tensorflow as tf

from data_generator import DataGenerator
from train_test_utils import model_evaluate, model_fit
from model import ASRModel

In [ ]:
print(tf.__version__)

In [ ]:
tf.test.is_gpu_available()

### Paths

In [ ]:
train_path = "./LibriSpeech100/train/train_all/"
dev_path = "./LibriSpeech100/dev/dev_all/"
test_path = "./LibriSpeech100/test/test_all/"

### Create DataGenerator objects

In [ ]:
train_data = DataGenerator(train_path)
val_data = DataGenerator(dev_path)
test_data = DataGenerator(test_path)

### Build Model

In [ ]:
model = ASRModel()
model.build(input_shape = (None, None, 20))
optimizer = tf.keras.optimizers.Adam()

In [ ]:
model.summary()

### Checkpoint

In [ ]:
ckpt_dir = './training_checkpoints'
ckpt = tf.train.Checkpoint(optimizer=optimizer, model = model)
manager = tf.train.CheckpointManager(ckpt, ckpt_dir, max_to_keep = 2)    

In [ ]:
losses, accuracies, val_losses, val_acc = model_fit(model, optimizer, train_data, manager, val_ds = val_data)

### Restore Checkpoint and Test Model

In [ ]:
cpkt.restore(manager.latest_checkpoint)

_, acc = model_evaluate(model, test_data, test=True)

In [ ]:
print(acc)